### ETL Process using Pandas

In [1]:
import pandas as pd
import requests
import warnings
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
from config import DATABASE_URI
import scrape_data
import get_api_data
import numpy as np

### Data Extraction of Restaurants and Pizzas

**Importing all libraries**

In [2]:
# Reading the csv into pandas
file_path = "../Resources/Datafiniti_Pizza_Restaurants_and_the_Pizza_They_Sell_May19.csv"
pizza_data_raw = pd.read_csv(file_path)

In [3]:
# Reviewing the pandas dataframe
pizza_data_raw.head(5)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVz3Y-7h3D1zeR_xDAqm,2017-06-30T05:05:40Z,2019-05-01T15:43:09Z,4203 E Kiehl Ave,"Pizza,Restaurant,American restaurants,Pizza Pl...",Accommodation & Food Services,Sherwood,US,us/ar/sherwood/4203ekiehlave/-1051391616,34.832300,...,USD,"2018-05-01T04:25:37.197Z,2018-04-16T04:36:02.3...",NaN,Cheese Pizza,Shotgun Dans Pizza,72120,USD,0,25,AR
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,2019-05-01T15:27:50Z,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Accommodation & Food Services,Phoenix,US,us/az/phoenix/25ecamelbackrd/-727422936,33.509266,...,USD,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",NaN,Pizza Cookie,Sauce Pizza Wine,85012,USD,0,25,AZ
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,2019-05-01T12:52:25Z,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,us/oh/cincinnati/3703paxtonave/-619797122,39.144883,...,USD,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",a saucelessampcomma double cheese pizza with a...,Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,2019-05-01T12:52:20Z,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Accommodation & Food Services,Madison Heights,US,us/mi/madisonheights/30495johnrrd/-874863116,42.516669,...,USD,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",NaN,Small Pizza,Hungry Howies Pizza,48071,USD,25,40,MI
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,...,USD,2016-03-31T02:34:04Z,NaN,Pizza Sub,Spartan Pizzeria,21224,USD,0,25,MD


In [4]:
# viewing raw meta_data and dimension of the data set
print(pizza_data_raw.info())
print(pizza_data_raw.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
id                    10000 non-null object
dateAdded             10000 non-null object
dateUpdated           10000 non-null object
address               10000 non-null object
categories            10000 non-null object
primaryCategories     10000 non-null object
city                  10000 non-null object
country               10000 non-null object
keys                  10000 non-null object
latitude              10000 non-null float64
longitude             10000 non-null float64
menuPageURL           1679 non-null object
menus.amountMax       10000 non-null float64
menus.amountMin       10000 non-null float64
menus.currency        10000 non-null object
menus.dateSeen        10000 non-null object
menus.description     3718 non-null object
menus.name            10000 non-null object
name                  10000 non-null object
postalCode            9996 non-null object
priceRangeC

In [5]:
# viewing columns of the data set
pizza_data_raw.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'menuPageURL', 'menus.amountMax', 'menus.amountMin', 'menus.currency',
       'menus.dateSeen', 'menus.description', 'menus.name', 'name',
       'postalCode', 'priceRangeCurrency', 'priceRangeMin', 'priceRangeMax',
       'province'],
      dtype='object')

The type possible questions we are trying to analyze using this dataset are?

_1)How many different types of pizzas are available?_

_2)How many pizza restaurants are available?_

_3)Which is the cheapest type of pizza sold?_

_4)Which is the most expensive pizza sold?_

_5)which provice has the maximum number of pizza restaurants?_

_6)Which city has the maximum number of restaurants?_

_7)Which is the cheapest and most expesive province for pizza?_

_8)Which is the cheapest and most expensive city for pizza?_

_9)Which is the most popular and leasr popular pizza?_

_10)Which all restaurant offer vegg pizzas based on description?_

_11)Which all categories of pizzas's are popular?_

_12)Which days of the week certains pizzas's are availabe?_

_13)Are any of restaurants identified as best restaurants of the state?_

_14)Any correlation between between rating found between restaurants of every state against other restaurants of the same state?_

**In order to answer the above questions, folowing columns will be selected**

**id, address, city, address, postalcode, country, latitude, longitude, menus.amountMax, menus.amountMin, menus.name, menus.dateSeen, menus.description,  province**

Note: Amount is in USD.
      This dataset peratins to pizza locations from USA only.

### Data Extraction of Pizza's that are sold by Restaurants across USA.

### Data Cleaning

In [6]:
# Removing un-necessary columns
pizza_data_transformed = pizza_data_raw[['id', 'address', 'categories', 'city', 'postalCode', 'latitude', 'longitude', 'menus.amountMax', 'menus.amountMin', 'menus.name', 'menus.description', 'menus.dateSeen','province']]
pizza_data_transformed.head()

,id,address,categories,city,postalCode,latitude,longitude,menus.amountMax,menus.amountMin,menus.name,menus.description,menus.dateSeen,province
0,AVz3Y-7h3D1zeR_xDAqm,4203 E Kiehl Ave,"Pizza,Restaurant,American restaurants,Pizza Pl...",Sherwood,72120,34.832300,-92.183800,7.98,7.98,Cheese Pizza,NaN,"2018-05-01T04:25:37.197Z,2018-04-16T04:36:02.3...",AR
1,AVweGPFF_7pvs4fzAAzQ,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Phoenix,85012,33.509266,-112.073044,6.00,6.00,Pizza Cookie,NaN,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",AZ
2,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Cincinnati,45209,39.144883,-84.432685,6.49,6.49,Pizza Blanca,a saucelessampcomma double cheese pizza with a...,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",OH
3,AVwdX4psIN2L1WUfvJB1,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Madison Heights,48071,42.516669,-83.106630,5.99,5.99,Small Pizza,NaN,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",MI
4,AVwdaeTtkufWRAb55pSH,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Baltimore,21224,39.286630,-76.566984,5.49,5.49,Pizza Sub,NaN,2016-03-31T02:34:04Z,MD


In [11]:
# Checking the agrregate values for min and max menu amount for pizza stores
aggregations = { 'menus.amountMax' : ["mean" ,"median",  "std" , "max" , "min"],
                'menus.amountMin' : ["mean", "median", "std" , "max" , "min"] }
pizza_data_transformed.groupby('id').agg(aggregations)
                    


menus.amountMax                                  \
                                mean  median       std    max    min   
id                                                                     
AV0BBqimLD2H7whiYWKZ       15.000000  15.000  2.828427  17.00  13.00   
AV0BCLdu-gnIPe8DUvU5        7.500000   7.500       NaN   7.50   7.50   
AV0BH9GWFcQ3k02bDmLF       12.365000  12.365  3.712311  14.99   9.74   
AV0H5evu3D1zeR_xDOVo        8.950000   8.950       NaN   8.95   8.95   
AV0HOMEC3D1zeR_xDOLw       10.904286   9.990  4.983783  21.99   7.59   
...                              ...     ...       ...    ...    ...   
AWIMWzSwa4HuVbedO3SQ       11.750000  11.750  8.131728  17.50   6.00   
AWIaO0F1a4HuVbedO4te        7.990000   7.990       NaN   7.99   7.99   
AWLTf5x7a4HuVbedPhcd       14.000000  14.000       NaN  14.00  14.00   
AWM7PB2fa4HuVbedQAtn        9.950000   9.950       NaN   9.95   9.95   
AWMaIedy3-Khe5l_iAhu        9.750000  10.000  3.774917  14.00   5.00   

                     menus.amountMin                                  
                                mean  median       std    max    min  
id                                                                    
AV0BBqimLD2H7whiYWKZ       15.000000  15.000  2.828427  17.00  13.00  
AV0BCLdu-gnIPe8DUvU5        7.500000   7.500       NaN   7.50   7.50  
AV0BH9GWFcQ3k02bDmLF       12.365000  12.365  3.712311  14.99   9.74  
AV0H5evu3D1zeR_xDOVo        8.950000   8.950       NaN   8.95   8.95  
AV0HOMEC3D1zeR_xDOLw       10.904286   9.990  4.983783  21.99   7.59  
...                              ...     ...       ...    ...    ...  
AWIMWzSwa4HuVbedO3SQ       11.750000  11.750  8.131728  17.50   6.00  
AWIaO0F1a4HuVbedO4te        7.990000   7.990       NaN   7.99   7.99  
AWLTf5x7a4HuVbedPhcd       14.000000  14.000       NaN  14.00  14.00  
AWM7PB2fa4HuVbedQAtn        9.950000   9.950       NaN   9.95   9.95  
AWMaIedy3-Khe5l_iAhu        9.750000  10.000  3.774917  14.00   5.00  

[2285 rows x 10 columns]

In [14]:
# Checking descriptive stats info on the data

pizza_data_transformed.describe()

,latitude,longitude,menus.amountMax,menus.amountMin
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,38.371895,-90.063800,14.032670,13.416902
std,4.573537,17.340379,17.607233,10.742678
min,21.421326,-157.802443,0.000000,0.000000
25%,34.422946,-104.801126,8.490000,8.000000
50%,40.118880,-82.909949,13.390000,12.990000
75%,40.905048,-75.194741,17.950000,16.990000
max,64.850500,-71.946170,1395.000000,243.000000


### Handling Outliers

In [15]:
# Based on the above data, will treat any amount above 50$ as outliers for maximum menu amount min menu amout
# and 0 as an outlier for minimum menu amout as it doesn't make sense for pizza menu amount to be zero in the menu.
# This is not transactional data but the details in the menu and so replacing the said outliers with mean values.
# Changing menus maximum amount
mean_amount_max = np.mean(pizza_data_transformed['menus.amountMax'])
pizza_data_transformed['menus.amountMax'] = pizza_data_transformed['menus.amountMax'].apply(lambda x:  mean_amount_max if x > 50 else x )
# Changing menu minimum amount
mean_amount_min = np.mean(pizza_data_transformed['menus.amountMin'])

pizza_data_transformed['menus.amountMin'] = pizza_data_transformed['menus.amountMin'].apply(lambda x: mean_amount_min if x == 0 else x)
pizza_data_transformed['menus.amountMin'] = pizza_data_transformed['menus.amountMin'].apply(lambda x: mean_amount_min if x > 50 else x)

In [16]:
# Checking descriptive stats info on the data again

pizza_data_transformed.describe()

,latitude,longitude,menus.amountMax,menus.amountMin
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,38.371895,-90.063800,13.395157,12.986676
std,4.573537,17.340379,6.620354,6.365608
min,21.421326,-157.802443,0.000000,0.250000
25%,34.422946,-104.801126,8.490000,8.247500
50%,40.118880,-82.909949,13.390000,12.990000
75%,40.905048,-75.194741,17.892500,16.950000
max,64.850500,-71.946170,49.990000,49.990000


In [17]:
# checking the na detailes within the columns
pizza_data_transformed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
id                   10000 non-null object
address              10000 non-null object
categories           10000 non-null object
city                 10000 non-null object
postalCode           9996 non-null object
latitude             10000 non-null float64
longitude            10000 non-null float64
menus.amountMax      10000 non-null float64
menus.amountMin      10000 non-null float64
menus.name           10000 non-null object
menus.description    3718 non-null object
menus.dateSeen       10000 non-null object
province             10000 non-null object
dtypes: float64(4), object(9)
memory usage: 1015.8+ KB


In [18]:
# menus.description    3718 non-null object. Removing all records that have NaN values in menus.description.
pizza_data_transformed = pizza_data_transformed.dropna()
pizza_data_transformed.reset_index(drop=True, inplace=True)
pizza_data_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 13 columns):
id                   3717 non-null object
address              3717 non-null object
categories           3717 non-null object
city                 3717 non-null object
postalCode           3717 non-null object
latitude             3717 non-null float64
longitude            3717 non-null float64
menus.amountMax      3717 non-null float64
menus.amountMin      3717 non-null float64
menus.name           3717 non-null object
menus.description    3717 non-null object
menus.dateSeen       3717 non-null object
province             3717 non-null object
dtypes: float64(4), object(9)
memory usage: 377.6+ KB


In [19]:
pizza_data_transformed.head(1)

,id,address,categories,city,postalCode,latitude,longitude,menus.amountMax,menus.amountMin,menus.name,menus.description,menus.dateSeen,province
0,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Cincinnati,45209,39.144883,-84.432685,6.49,6.49,Pizza Blanca,a saucelessampcomma double cheese pizza with a...,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",OH


In [20]:
# Capitalizing menus.description column.
pizza_data_transformed['menus.description'] = pizza_data_transformed['menus.description'].apply(lambda x: x.capitalize())
pizza_data_transformed['menus.description'][1]

'Prosciutto arugula and shaved parmesan drizzled with balsamic glaze'

In [21]:
# Converting the case of the menus.name column to Title case.
pizza_data_transformed['menus.name'] = pizza_data_transformed['menus.name'].apply(lambda x : x.title())
pizza_data_transformed['menus.name'].value_counts().sort_values(ascending=False)[:5]

White Pizza          50
Margherita Pizza     49
Bbq Chicken Pizza    49
Hawaiian Pizza       47
Veggie Pizza         39
Name: menus.name, dtype: int64

In [22]:
# Removing duplicated rows
pizza_data_transformed = pizza_data_transformed.drop_duplicates(keep='last')

In [23]:
# Checking for duplicated rows
pizza_data_transformed[pizza_data_transformed.duplicated()].count()
pizza_data_transformed['id'].value_counts()[:5]

AVweE6g9kufWRAb5_ykR    51
AVwcw735IN2L1WUfoo1p    44
AVzBGRKZFcQ3k02bBecq    40
AVzBBC1w3D1zeR_xBEbJ    39
AVzA1eJAFcQ3k02bBXX0    39
Name: id, dtype: int64

### Data Transforming Restaurant Data

In [24]:
# Separating pizza_data and restaurant data into separate data frames
restaurant_transformed = pizza_data_transformed[['id','address', 'postalCode', 'latitude', 'longitude', 'city','province', 'categories']].drop_duplicates('id', keep='last').reset_index(drop=True)
restaurant_transformed.head()


,id,address,postalCode,latitude,longitude,city,province,categories
0,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,45209,39.144883,-84.432685,Cincinnati,OH,"Restaurant,Pizza Place,Restaurants"
1,AVwchRVZkufWRAb5wDIV,4061 28th St SE,49512,42.912878,-85.566939,Grand Rapids,MI,"Restaurant,Italian Restaurant,Pizza Place,Ital..."
2,AVwc7s1wIN2L1WUfqehD,146 N Glendora Ave,91741,34.137502,-117.865331,Glendora,CA,"Pizza,Restaurant,Pizza Place"
3,AVwc2xOokufWRAb5zu4_,965 N Resler Dr,79912,31.862409,-106.558431,El Paso,TX,"Restaurant,Pizza Place,Restaurants"
4,AVwdixeC_7pvs4fz67Pw,1584 N Woodland Blvd,32720,29.046703,-81.303817,DeLand,FL,"Restaurant,Pizza Place,Restaurants"


In [25]:
# Writing transformed restaurant data to a csv
restaurant_transformed = restaurant_transformed.rename(columns={'postalCode': 'zip_code', 'province': 'state'})
restaurant_transformed.head()

,id,address,zip_code,latitude,longitude,city,state,categories
0,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,45209,39.144883,-84.432685,Cincinnati,OH,"Restaurant,Pizza Place,Restaurants"
1,AVwchRVZkufWRAb5wDIV,4061 28th St SE,49512,42.912878,-85.566939,Grand Rapids,MI,"Restaurant,Italian Restaurant,Pizza Place,Ital..."
2,AVwc7s1wIN2L1WUfqehD,146 N Glendora Ave,91741,34.137502,-117.865331,Glendora,CA,"Pizza,Restaurant,Pizza Place"
3,AVwc2xOokufWRAb5zu4_,965 N Resler Dr,79912,31.862409,-106.558431,El Paso,TX,"Restaurant,Pizza Place,Restaurants"
4,AVwdixeC_7pvs4fz67Pw,1584 N Woodland Blvd,32720,29.046703,-81.303817,DeLand,FL,"Restaurant,Pizza Place,Restaurants"


In [26]:
# Writing the transformed data to a csv
restaurant_transformed.to_csv("../Resources/restaurant.csv", index=False)

**Gathering additiona information from Google API for restaurant data**


In [27]:
# collecting data from google api
restaurant_transformed = get_api_data.get_from_api()

Address: 3703 Paxton Ave, Cincinnati, OH 45209, USA Name: Mio's Pizzeria Rating: 3.7 Price Level: 2
------------
Address: 4061 28th St SE, Grand Rapids, MI 49512, USA Name: Buddy's Pizza Rating: 4.4 Price Level: 2
------------
Address: 965 N Resler Dr, El Paso, TX 79912, USA Name: Little Caesars Pizza Rating: 3.4 Price Level: 1
------------
Missing field/result... skipping.
Address: 1477 York Ave, New York, NY 10075, USA Name: Salvo's Pizzabar Rating: 4.4 Price Level: 1
------------
Address: 786 Main St, Poughkeepsie, NY 12603, USA Name: Tony's Pizza Pit Rating: 3.6 Price Level: 1
------------
Address: 1612 Franklin St, Michigan City, IN 46360, USA Name: Albano's Villa Rating: 4.3 Price Level: 2
------------
Address: 2300 Muscatine Ave, Iowa City, IA 52240, USA Name: Papa Murphy's Take 'N' Bake Pizza Rating: 4.5 Price Level: 1
------------
Address: 9705 Mintworth Ave, Charlotte, NC 28227, USA Name: Angela's Pizza & Restaurant Rating: 4.4 Price Level: 2
------------
Missing field/result

Address: 1431 E Williams Field Rd, Gilbert, AZ 85295, USA Name: Barro's Pizza Rating: 4.4 Price Level: 1
------------
Address: 104 W Girard Ave, Philadelphia, PA 19123, USA Name: Paradise Restaurant Rating: 4.3 Price Level: 2
------------
Address: 3210 E Colonial Dr, Orlando, FL 32803, USA Name: Pizzeria Del-Dio Italian Restaurant Rating: 4.6 Price Level: 1
------------
Address: 831 7th Ave, New York, NY 10019, USA Name: Famous Original Rays Pizza Rating: 3.7 Price Level: 1
------------
Address: 6042 SE Division St, Portland, OR 97206, USA Name: Pizzicato Pizza Rating: 4.3 Price Level: 2
------------
Address: 936 2nd Ave, New York, NY 10022, USA Name: Abitino's Pizzeria Rating: 3.4 Price Level: 2
------------
Address: 691 N High St, Columbus, OH 43215, USA Name: Fabian's Rating: 4.2 Price Level: 2
------------
Address: 180 S Glendora Ave, West Covina, CA 91790, USA Name: Bun Street Rating: 4.7 Price Level: 2
------------
Address: 530 N Progress Ave, Harrisburg, PA 17109, USA Name: Toni

Address: 517 Warren St, Hudson, NY 12534, USA Name: Baba Louie's Rating: 4.6 Price Level: 2
------------
Address: 731 Lincoln Rd, Miami Beach, FL 33139, USA Name: Spris Rating: 4.4 Price Level: 2
------------
Missing field/result... skipping.
Address: 800 Nicollet Mall, Minneapolis, MN 55402, USA Name: McCormick & Schmick's Seafood & Steaks Rating: 4.2 Price Level: 3
------------
Address: 800 Nicollet Mall, Minneapolis, MN 55402, USA Name: Greenfield Natural Kitchen - Nicollet Mall Rating: 3.8 Price Level: 1
------------
Address: 165 Bleecker St, New York, NY 10012, USA Name: Fiore's Pizza Rating: 4.2 Price Level: 1
------------
Address: 1975 Magwood Dr, Charleston, SC 29414, USA Name: Bushido Japanese Restaurant Rating: 4.2 Price Level: 2
------------
Address: 1975 Magwood Dr, Charleston, SC 29414, USA Name: Baroni's New York Pizza Rating: 3.9 Price Level: 2
------------
Address: 1975 Magwood Dr, Charleston, SC 29414, USA Name: Rio Grande Rating: 4.3 Price Level: 1
------------
Addres

Address: 1320 Mockingbird Ln, Sulphur Springs, TX 75482, USA Name: Pizza Inn Rating: 4.4 Price Level: 1
------------
Address: 108 Franklin St NE, Olympia, WA 98501, USA Name: Old School Pizzeria Rating: 4.5 Price Level: 1
------------
Address: 128 W Lancaster Ave, Wayne, PA 19087, USA Name: Domino's Pizza Rating: 3.3 Price Level: 1
------------
Address: 2283 W Market St, Akron, OH 44313, USA Name: Romeo's Pizza Rating: 4.3 Price Level: 2
------------
Address: 6014 Radio Rd, Naples, FL 34104, USA Name: Pizza Steve's Rating: 4.5 Price Level: 2
------------
Address: 1706 University Ave, Berkeley, CA 94703, USA Name: West Coast Pizza Rating: 4 Price Level: 1
------------
Address: 2507 South Rd, Poughkeepsie, NY 12601, USA Name: La Luna Pizza & Restaurant Rating: 3.4 Price Level: 2
------------
Address: 2507 South Rd, Poughkeepsie, NY 12601, USA Name: Five Guys Rating: 4.5 Price Level: 2
------------
Address: 2450 Crystal Dr, Arlington, VA 22202, USA Name: Mezeh Mediterranean Grill (Crystal

Address: 300 S Roosevelt Dr, Seaside, OR 97138, USA Name: Domino's Pizza Rating: 4.1 Price Level: 1
------------
Address: 2720 McCullough Ave, San Antonio, TX 78212, USA Name: Barbaro Rating: 4.4 Price Level: 2
------------
Address: 181 E Broadway, New York, NY 10002, USA Name: Lower East Side Pizza Rating: 3.4 Price Level: 2
------------
Missing field/result... skipping.
Address: 5924 S Land Park Dr, Sacramento, CA 95822, USA Name: Giovanni's Old World New York Pizzeria Rating: 4.1 Price Level: 2
------------
Address: 27080 John R Rd, Madison Heights, MI 48071, USA Name: Happy's Pizza Rating: 3.2 Price Level: 2
------------
Address: 214 Court St, Brooklyn, NY 11201, USA Name: Layla Jones Rating: 4.3 Price Level: 1
------------
Address: 835 Seven Hills Dr, Henderson, NV 89052, USA Name: Angelina's Pizzeria Rating: 3.9 Price Level: 2
------------
Address: 401 C St, San Diego, CA 92101, USA Name: A Brooklyn Pizzeria Rating: 4.2 Price Level: 2
------------
Missing field/result... skipping

Missing field/result... skipping.
Address: 219 W Maple Ave, Fayetteville, WV 25840, USA Name: Pies & Pints - Fayetteville, WV Rating: 4.7 Price Level: 2
------------
Address: 538 Warren St, Huntington, IN 46750, USA Name: Z-Place Pizza Rating: 4.3 Price Level: 1
------------
Address: 2600 Ardmore Blvd, Pittsburgh, PA 15221, USA Name: Jodi B's Restaurant Rating: 4.3 Price Level: 1
------------
Address: 2600 Ardmore Blvd, Pittsburgh, PA 15221, USA Name: Domino's Pizza Rating: 2.6 Price Level: 1
------------
Missing field/result... skipping.
Address: 5230 Baymeadows Rd, Jacksonville, FL 32217, USA Name: Papa John's Pizza Rating: 3.6 Price Level: 1
------------
Address: 3675 J St, Sacramento, CA 95816, USA Name: Juno's Kitchen & Delicatessen Rating: 4.7 Price Level: 2
------------
Address: 140 Crescent Dr, Pleasant Hill, CA 94523, USA Name: Zachary's Chicago Pizza Rating: 4.6 Price Level: 2
------------
Missing field/result... skipping.
Address: 4085 Atlantic Ave, Long Beach, CA 90807, USA

Address: 320 Water St, Eau Claire, WI 54703, USA Name: Gyro King Rating: 4.6 Price Level: 1
------------
Missing field/result... skipping.
Address: 438 S York Rd, Bensenville, IL 60106, USA Name: Mamma Maria's Pizzeria & Ristorante Rating: 4.4 Price Level: 2
------------
Address: 4938 S Staples St, Corpus Christi, TX 78411, USA Name: Subway Restaurants Rating: 3.6 Price Level: 1
------------
Missing field/result... skipping.
Missing field/result... skipping.
Address: 2030 New Bern Ave, Raleigh, NC 27610, USA Name: Frank's Pizza & Italian Restaurant Rating: 4.5 Price Level: 1
------------
Missing field/result... skipping.
Address: 2717 Vine St, Cincinnati, OH 45219, USA Name: LaRosa's Pizza Clifton Rating: 3.8 Price Level: 2
------------
Address: 3174 NW Federal Hwy, Jensen Beach, FL 34957, USA Name: Sbarro Rating: 2.8 Price Level: 1
------------
Missing field/result... skipping.
Address: 1700 W New Haven Ave, Melbourne, FL 32904, USA Name: Outback Steakhouse Rating: 4.2 Price Level: 2


Address: 41070 California Oaks Rd, Murrieta, CA 92562, USA Name: Chili's Grill & Bar Rating: 4.2 Price Level: 2
------------
Address: 28 West St, New York, NY 10004, USA Name: Inatteso Pizzabar Casano Rating: 4.4 Price Level: 2
------------
Address: 260 S 200 W, Salt Lake City, UT 84101, USA Name: Settebello Rating: 4.5 Price Level: 2
------------
Address: 742 N State St, Westerville, OH 43082, USA Name: Classic Pizza Rating: 4.3 Price Level: 1
------------
Address: 6171 Mission Gorge Rd, San Diego, CA 92120, USA Name: Souplantation Rating: 4.4 Price Level: 1
------------
Address: 6171 Mission Gorge Rd, San Diego, CA 92120, USA Name: Subway Rating: 3.8 Price Level: 1
------------
Address: 2709 McKinney Ave, Dallas, TX 75204, USA Name: LA Gourmet Pizza Rating: 4.1 Price Level: 2
------------
Address: 380 N Park Rd, Hollywood, FL 33021, USA Name: Big Louie's Pizza Rating: 3.9 Price Level: 2
------------
Address: 2650 King Rd, Frisco, TX 75034, USA Name: Papa John's Pizza Rating: 3.4 Pric

Address: 4880 Lower Roswell Rd, Marietta, GA 30068, USA Name: Taco Bell Rating: 3.7 Price Level: 1
------------
Address: 2553 S Colorado Blvd, Denver, CO 80222, USA Name: Subway Rating: 4.5 Price Level: 1
------------
Address: 4900 Dodge St, Omaha, NE 68132, USA Name: Varsity Sports Cafe and Roman Coin Pizza - Dundee Rating: 4 Price Level: 2
------------
Address: 105 E Main St, Bartow, FL 33830, USA Name: Palace Pizza Rating: 4.7 Price Level: 1
------------
Address: 4120 Woodward Ave, Detroit, MI 48201, USA Name: Sgt. Pepperoni's Pizzeria & Deli Rating: 4.1 Price Level: 1
------------
Missing field/result... skipping.
Address: 301 Main St, Huntington Beach, CA 92648, USA Name: Legend's Sports Bar Rating: 4.2 Price Level: 2
------------
Address: 1700 W 11th Ave, Eugene, OR 97402, USA Name: Papa's Pizza West 11th Rating: 4.1 Price Level: 2
------------
Address: 111 S Broadway, Rochester, MN 55904, USA Name: zpizza Rating: 3.6 Price Level: 2
------------
Address: 1230 Sunset Blvd, Rocklin

In [28]:
# Viewing the new data
restaurant_transformed.head()

,id,address,zip_code,latitude,longitude,city,state,categories,rating,name,price_level
0,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,45209,39.144883,-84.432685,Cincinnati,OH,"Restaurant,Pizza Place,Restaurants",3.7,Mio's Pizzeria,2
1,AVwchRVZkufWRAb5wDIV,4061 28th St SE,49512,42.912878,-85.566939,Grand Rapids,MI,"Restaurant,Italian Restaurant,Pizza Place,Ital...",4.4,Buddy's Pizza,2
2,AVwc7s1wIN2L1WUfqehD,146 N Glendora Ave,91741,34.137502,-117.865331,Glendora,CA,"Pizza,Restaurant,Pizza Place",,,
3,AVwc2xOokufWRAb5zu4_,965 N Resler Dr,79912,31.862409,-106.558431,El Paso,TX,"Restaurant,Pizza Place,Restaurants",3.4,Little Caesars Pizza,1
4,AVwdixeC_7pvs4fz67Pw,1584 N Woodland Blvd,32720,29.046703,-81.303817,DeLand,FL,"Restaurant,Pizza Place,Restaurants",,,


**Writing this new restaurant data to a csv**

In [29]:
# Writing the new restaurant data to Resources folder
restaurant_transformed.to_csv("../Resources/restaurant.csv",index=None)
restaurant_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 11 columns):
id             980 non-null object
address        980 non-null object
zip_code       980 non-null object
latitude       980 non-null float64
longitude      980 non-null float64
city           980 non-null object
state          980 non-null object
categories     980 non-null object
rating         980 non-null object
name           980 non-null object
price_level    980 non-null object
dtypes: float64(2), object(9)
memory usage: 84.3+ KB


**Replace all blank values with np.nan and fixing Nan values**

In [30]:
# Viewing the null data
print(f" Total null values {restaurant_transformed.isnull().sum()}")
restaurant_transformed = restaurant_transformed.replace('', np.nan, regex=True)
restaurant_transformed.info()

 Total null values id             0
address        0
zip_code       0
latitude       0
longitude      0
city           0
state          0
categories     0
rating         0
name           0
price_level    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 11 columns):
id             980 non-null object
address        980 non-null object
zip_code       980 non-null object
latitude       980 non-null float64
longitude      980 non-null float64
city           980 non-null object
state          980 non-null object
categories     980 non-null object
rating         503 non-null float64
name           503 non-null object
price_level    503 non-null float64
dtypes: float64(4), object(7)
memory usage: 84.3+ KB


In [31]:
# Viewing the null data
# Replacing all null values with -NA- for restaurant names

restaurant_transformed['name'] = restaurant_transformed['name'].fillna('-NA-')
# Replacing all null values with average of rating for restaurant rating
restaurant_transformed['rating'] = restaurant_transformed['rating'].fillna(round(restaurant_transformed['rating'].mean(),1))

# Replacing all null values with average of price_level for restaurant price level
restaurant_transformed['price_level'] = restaurant_transformed['price_level'].fillna(round(restaurant_transformed['price_level'].mean(),1))




In [32]:
# Verifying if Nan are replaced
print(restaurant_transformed.info())
restaurant_transformed.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 11 columns):
id             980 non-null object
address        980 non-null object
zip_code       980 non-null object
latitude       980 non-null float64
longitude      980 non-null float64
city           980 non-null object
state          980 non-null object
categories     980 non-null object
rating         980 non-null float64
name           980 non-null object
price_level    980 non-null float64
dtypes: float64(4), object(7)
memory usage: 84.3+ KB
None


,id,address,zip_code,latitude,longitude,city,state,categories,rating,name,price_level
0,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,45209,39.144883,-84.432685,Cincinnati,OH,"Restaurant,Pizza Place,Restaurants",3.7,Mio's Pizzeria,2.0
1,AVwchRVZkufWRAb5wDIV,4061 28th St SE,49512,42.912878,-85.566939,Grand Rapids,MI,"Restaurant,Italian Restaurant,Pizza Place,Ital...",4.4,Buddy's Pizza,2.0
2,AVwc7s1wIN2L1WUfqehD,146 N Glendora Ave,91741,34.137502,-117.865331,Glendora,CA,"Pizza,Restaurant,Pizza Place",4.2,-NA-,1.6
3,AVwc2xOokufWRAb5zu4_,965 N Resler Dr,79912,31.862409,-106.558431,El Paso,TX,"Restaurant,Pizza Place,Restaurants",3.4,Little Caesars Pizza,1.0
4,AVwdixeC_7pvs4fz67Pw,1584 N Woodland Blvd,32720,29.046703,-81.303817,DeLand,FL,"Restaurant,Pizza Place,Restaurants",4.2,-NA-,1.6


### Data Transforming of Pizza Data

In [33]:
# Creating a pizza table keep unique records of restaurant id and menus.name
pizza_transformed = pizza_data_transformed[['id', 'menus.name', 'menus.description', 'menus.amountMax', 'menus.amountMin', 'menus.dateSeen']].drop_duplicates(['id','menus.name'],keep='last').reset_index(drop=True)


In [34]:
# Renaming the columns for pizza
pizza_transformed = pizza_transformed.rename(columns={'id':'restaurant_id', 'menus.name':'pizza_name' \
                              , 'menus.description':'pizza_description', 'menus.amountMax': 'maximum_amount',
                              'menus.amountMin':'minimum_amount', 'menus.dateSeen' : 'pizza_dates'})
pizza_transformed.head()

,restaurant_id,pizza_name,pizza_description,maximum_amount,minimum_amount,pizza_dates
0,AVwdRGa9_7pvs4fz4E3K,Pizza Blanca,A saucelessampcomma double cheese pizza with a...,6.49,6.49,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4..."
1,AVwchRVZkufWRAb5wDIV,Prosciutto Arugula Pizza,Prosciutto arugula and shaved parmesan drizzle...,15.92,15.92,"2018-04-26T21:25:35.237Z,2018-06-05T01:35:19.2..."
2,AVwc7s1wIN2L1WUfqehD,Three Cheese Pizza,Topped with mozzarellaampcomma feta and ricotta,11.50,11.50,2018-05-11T22:26:10.666Z
3,AVwc7s1wIN2L1WUfqehD,Pizza Sandwich,Includes 1 toppingampcomma additional toppings...,7.00,7.00,2018-05-11T22:26:10.666Z
4,AVwc2xOokufWRAb5zu4_,Hotnready Large Pizza,Pepperoni or cheese original round no limit ca...,5.00,5.00,"2018-05-14T18:17:05.194Z,2018-04-02T12:17:58.5..."


In [35]:
# Writing pizza data as a CSV file.
pizza_data_transformed.to_csv('../Resources/pizza.csv')

### Collecting State wise Best Restaurants Data Across USA.

In [36]:
# Extracting statewise best pizza stores data using splinter and bs4
best_pizza_stores_original = scrape_data.scrape_best_pizza()

Alabama---Bottega Café---Birmingham
Alaska---Moose’s Tooth Pub and Pizzeria---Anchorage
Arizona---Pizzeria Bianco---Phoenix
Arkansas---Wood Stone Craft Pizza & Bar---Fayetteville
California---Tony’s Pizza Napoletana---San Francisco
Colorado---Pizzeria Locale---Denver
Connecticut---Frank Pepe Pizzeria Napoletana---New Haven
Delaware---The Wood Fired Pizza Shop---Newark
Florida---Scuola Vecchia Pizza e Vino---Delray Beach
Georgia---O4W Pizza---Duluth
Hawaii---J.J. Dolan’s---Honolulu
Idaho---Guido’s Original New York Style Pizza---Boise
Illinois---Pequod’s---Chicago
Indiana---Diavola---Indianapolis
Iowa---Gusto Pizza Co.---Des Moines
Kansas---AJ’s NY Pizzeria---Topeka
Kentucky---Garage Bar---Louisville
Louisiana---Pizza Domenica---New Orleans
Maine---Micucci Grocery Store---Portland
Maryland---Inferno---Darnestown
Massachusetts---Area Four---Cambridge
Michigan---Buddy’s Pizza---Detroit
Minnesota---Pizzeria Lola---Minneapolis
Mississippi---TriBecca Allie Café---Sardis
Missouri---Imo’s---St

In [37]:
# Viewing best pizza stores data
best_pizza_stores_original.head()

,restaurant_name,city,state
0,Bottega Café,Birmingham,Alabama
1,Moose’s Tooth Pub and Pizzeria,Anchorage,Alaska
2,Pizzeria Bianco,Phoenix,Arizona
3,Wood Stone Craft Pizza & Bar,Fayetteville,Arkansas
4,Tony’s Pizza Napoletana,San Francisco,California


In [38]:
# Writing the dataframe to a csv
best_pizza_stores_original.to_csv("../Resources/best_pizza_stores.csv", index=None)


**Gathering additional info like rating, price_level from Google API**

In [39]:
best_pizza_stores_transformed = get_api_data.get_best_from_api()

Address: 2240 Highland Ave, Birmingham, AL 35205, USA Name: Bottega Cafe Rating: 4.7 Price Level: 3
------------
Address: 3300 Old Seward Hwy, Anchorage, AK 99503, USA Name: Moose's Tooth Pub & Pizzeria Rating: 4.7 Price Level: 2
------------
Address: 623 E Adams St, Phoenix, AZ 85004, USA Name: Pizzeria Bianco Rating: 4.4 Price Level: 2
------------
Address: 3619 Mall Ave, Fayetteville, AR 72703, USA Name: Wood Stone Craft Pizza Uptown Rating: 4.4 Price Level: 2
------------
Address: 1570 Stockton St, San Francisco, CA 94133, USA Name: Tony's Pizza Napoletana Rating: 4.5 Price Level: 2
------------
Address: 550 Broadway, Denver, CO 80203, USA Name: Pizzeria Locale Rating: 4.6 Price Level: 1
------------
Address: 157 Wooster St, New Haven, CT 06511, USA Name: Frank Pepe Pizzeria Napoletana Rating: 4.6 Price Level: 2
------------
Address: 140 E Cleveland Ave, Newark, DE 19711, USA Name: The Wood Fired Pizza Shop Rating: 4.8 Price Level: 2
------------
Address: 522 E Atlantic Ave, Delray

**Replacing all blank values with np.nan and fixing Nan values**

In [40]:
# Viewing the updated restaurant data.
print(f" Total null values  {best_pizza_stores_transformed.isnull().sum()}")

best_pizza_stores_transformed = best_pizza_stores_transformed.replace('', np.nan, regex=True)

best_pizza_stores_transformed.info()


 Total null values  restaurant_name    0
city               0
state              0
rating             0
address            0
price_level        0
zip_code           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
restaurant_name    50 non-null object
city               50 non-null object
state              50 non-null object
rating             41 non-null float64
address            41 non-null object
price_level        41 non-null float64
zip_code           41 non-null object
dtypes: float64(2), object(5)
memory usage: 2.9+ KB


In [41]:
# Fixing NaN values
# Replacing all null values with -NA- for restaurant names
best_pizza_stores_transformed['address'] = best_pizza_stores_transformed['address'].fillna('-NA-')
# Replacing all null values with average of rating for restaurant rating
best_pizza_stores_transformed['rating'] = best_pizza_stores_transformed['rating'].fillna(round(best_pizza_stores_transformed['rating'].mean(),1))
# Replacing all null values with average of price_level for restaurant price level
best_pizza_stores_transformed['price_level'] = best_pizza_stores_transformed['price_level'].fillna(round(best_pizza_stores_transformed['price_level'].mean(),1))


In [42]:
# Viewing the new data.
best_pizza_stores_transformed.head()

,restaurant_name,city,state,rating,address,price_level,zip_code
0,Bottega Café,Birmingham,Alabama,4.7,2240 Highland Ave,3.0,35205
1,Moose’s Tooth Pub and Pizzeria,Anchorage,Alaska,4.7,3300 Old Seward Hwy,2.0,99503
2,Pizzeria Bianco,Phoenix,Arizona,4.4,623 E Adams St,2.0,85004
3,Wood Stone Craft Pizza & Bar,Fayetteville,Arkansas,4.4,3619 Mall Ave,2.0,72703
4,Tony’s Pizza Napoletana,San Francisco,California,4.5,1570 Stockton St,2.0,94133


In [43]:
#  Writing to transformed data to csv.
best_pizza_stores_transformed.to_csv("../Resources/best_pizza_stores.csv", index=None)

### Create database connection

**Connect to local database**

In [44]:
# DATABASE_URI is in the format postgresql://<username>:<password>@localhost:5432/pizza_db
engine = create_engine(DATABASE_URI)

**Check for tables**

In [45]:
engine.table_names()

['restaurant', 'pizza', 'best_pizza_restaurants']

### Use pandas to load csv converted DataFrame into database

**Loading restaurant data into postgreSQL table**

In [46]:
restaurant_transformed.to_sql(name='restaurant', if_exists='append', con=engine, index=False)

**Loading pizza data into postgreSQL table**

In [47]:
pizza_transformed.to_sql(name='pizza', if_exists='append', con=engine, index=False)


**Loading best_pizza_restaurants data into postgreSQL table**

In [48]:
best_pizza_stores_transformed.to_sql(name='best_pizza_restaurants', if_exists='append', con=engine, index=False)
